# Setup

## Import dependencies

In [ ]:
import mne
from mne.time_frequency import tfr_multitaper
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patchesw
import pickle
import pandas as pd
from scipy import signal
from scipy import integrate
from scipy.stats import f_oneway
import pywt
import csv
mpl.rcParams['agg.path.chunksize'] = 10000
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
import matplotlib.ticker as ticker
import matplotlib.lines as mlines
from scipy.integrate import simps
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import ttest_ind, kruskal
from itertools import combinations
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats
import statsmodels.stats.multicomp as mc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.metrics import auc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import itertools

## Define analysis and plotting functions

In [ ]:
def group_eeg_processing(group,
                         srate,
                         epoch_scheme,
                         low_cutoff=1,
                         high_cutoff=200,
                         notch_width=1,
                         denoise=True,
                         filter_type="mne",
                         denoise_extent_threshold=1):
    """
    workhorse function for loading group EEG data and processing (applying bandpass and notch filters, then: 
            - for single_epoch: applying time-frequency response (tfr) analysis with various baseline 
            corrections
            - for multi_epoch: setting epochs at a few experimentally relevant timepoints and computing power 
            spectral density in those epochs
            
    params::
    group: dictionary containing keys "label" and "file_paths"
    srate: eeg sampling rate
    epoch_scheme: "single_epoch" or "multi_epoch" analysis
    low_cutoff: bandpass filter low end
    high_cutoff: bandpass filter high end
    notch_width: notch filter width (only used for filter type "mne")
    denoise: whether to apply wavelet-based denoising
    filter_type: type of notch/bandpass filters to apply (can be "jeff" or "mne")
    denoise_extent_threshold: how much denoising is too much, in units of 
        raw data std deviation? 
    """
    count = 0
    psds=[]
    psd_dbs=[]
    tfrs=[]
    tfrs_logratio=[]
    tfrs_mean=[]
    tfrs_ratio=[]
    tfrs_logratio=[]
    tfrs_mean=[]
    tfrs_zscore=[]
    tfrs_zlogratio=[]
    tfrs_percent=[]
    
    #quality control checks
    notch_filter_worked=[]
    denoising_shift_hist=[]
    denoising_shifts_above_thresh=[]
    
    for f in range(len(group["file_paths"])):
        # Get path to data
        print('Loading file:', group["file_paths"][f])
        df = pd.read_csv(group["file_paths"][f], usecols=[2], skiprows=5)
        raw_data = df.values.squeeze()
        raw_data = raw_data[np.newaxis, :]
        info = mne.create_info(["ch1"], sfreq = srate, ch_types=["eeg"])
        raw = mne.io.RawArray(raw_data, info)

        denoising_above1sd_change = 0
        
        # Filter - bandpass at selected low and high cutoffs 
        if filter_type=="mne":
            raw.filter(l_freq=low_cutoff, h_freq=high_cutoff)
            raw.notch_filter(freqs=(120), picks='eeg', 
                 method='spectrum_fit', filter_length='10s', notch_widths=notch_width)

        if denoise==True:
            print("\nDenoising data with Wavelet Decomposition")
            filtered_raw = raw.get_data()[0]
            
            # Multilevel wavelet decomposition with Daubechies 8 wavelet
            # number = granularity, lower number = lower granularity
            filtered_raw_decomposed = pywt.wavedec(filtered_raw,'db8', level=5) #chose level 5 for best
            
            filtered_denoised_decomposed = []
            for subband in filtered_raw_decomposed:
                med = np.median(np.abs(subband))
                tot = np.sum(np.abs(subband))
                subband[np.abs(subband)>3*med] = 0
                subband *= tot / np.sum(np.abs(subband))
                filtered_denoised_decomposed.append(subband)

            filtered_denoised_raw = pywt.waverec(filtered_denoised_decomposed,'db8')
            
            #Calculating how much the data was altered by the wavelet-based denoising
            filtered_noisy_sd = np.std(filtered_raw)
            length=np.min([len(filtered_denoised_raw),len(filtered_raw)])
            denoising_above_thresh_change = np.sum(np.abs(filtered_denoised_raw[0:length] - filtered_raw[0:length]) > denoise_extent_threshold*filtered_noisy_sd)/length
            hist, denoising_shift_hist_bins = np.histogram(((filtered_denoised_raw[0:length] - filtered_raw[0:length])/filtered_noisy_sd), bins=30, range=[0,3])
            denoising_shift_hist.append(hist)
            print("Denoising complete; " + str(denoising_above_thresh_change*100) + "% of data corrected by >" + str(denoise_extent_threshold) + "SD.\n")
            denoising_shifts_above_thresh.append(denoising_above_thresh_change*100)
            raw = mne.io.RawArray(filtered_denoised_raw[np.newaxis, :], info)
            
        else:
            # If denoising is not applied, put some dummy variables in that spot in dictionary output
            denoising_shift_hist.append(None)
            denoising_shift_hist_bins=None
                        
        if epoch_scheme=="single_epoch":
            # Make data into one big epoch for tfr / full session spectrum analysis
            events = np.array([[0, 0, 1]])
            raw_epoch = mne.Epochs(raw, events, event_id=1, tmin=0, tmax=44*60, #tmax = minutes x seconds
                                   baseline=None, preload=True) #tmax = mins*sec
            
            # Compute and append PSD to dictionary
            psd=raw_epoch.compute_psd(fmin=1, fmax=150, method="welch", n_per_seg=int(raw.info["sfreq"]))
            psd_freqs=psd.freqs
            psd=psd.get_data().squeeze()
            psds.append(psd)
            psd_db = 10 * np.log10(psd / (1e-6)**2)
            psd_dbs.append(psd_db)
            
            # get indices of 55 hz, 60 hz PSD value
            idx_55 = np.argmax(psd_freqs>55)
            idx_60 = np.argmax(psd_freqs>60)
            
            # test if power at 60 Hz is greater than power at 55 Hz
            if psd_db[idx_60]-psd_db[idx_55] > 0:
                notch_filter_worked.append(False)
            else:
                notch_filter_worked.append(True)
                
            count = count+1
    
            #compute tfr
            freqs = np.linspace(1, 55, 55)
            power = tfr_multitaper(raw_epoch, freqs=freqs, n_cycles=freqs / 2, use_fft=True,
                                   time_bandwidth=2, return_itc=False,
                                   decim=100, n_jobs=1)
            baseline_inds = [0, 60*5] #baseline as first 5 mins of recording
            tfrs.append(power)

            # Apply various kinds of baseline normalizations
            tfrs_logratio.append(power.copy().apply_baseline(baseline_inds,mode="logratio"))
            tfrs_mean.append(power.copy().apply_baseline(baseline_inds,mode="mean"))
            tfrs_zlogratio.append(power.copy().apply_baseline(baseline_inds,mode="zlogratio"))
            tfrs_zscore.append(power.copy().apply_baseline(baseline_inds,mode="zscore"))
            tfrs_percent.append(power.copy().apply_baseline(baseline_inds,mode="percent")*100) #multiply by 100 because its ratios
            
            print("DONE WITH FILE " + str(count) + " OF " + str(len(group["file_paths"])) + " FOR GROUP " + group["label"] + "\n\n")
            
    if epoch_scheme=="single_epoch":
        group["psd"]=psds
        group["psd_db"]=psd_dbs
        group["tfr"]=tfrs
        group["tfr_logratio"]=tfrs_logratio
        group["tfr_mean"]=tfrs_mean
        group["tfr_zlogratio"]=tfrs_zlogratio
        group["tfr_zscore"]=tfrs_zscore
        group["tfr_percent"]=tfrs_percent
        group["freqs"]=psd_freqs
        group["denoising_shift_hist"]=denoising_shift_hist
        group["denoising_shift_hist_bins"]=denoising_shift_hist_bins
        group["denoising_shift_above_thresh"]=denoising_shifts_above_thresh
        group["notch_filter_worked"]=notch_filter_worked

    group["epoch_scheme"]=epoch_scheme
    
    return group



def plot_spectrogram(groups,vmin,vmax,mode,cmap=None,figW=15,figH=2.5,save=True,output_dir=None):
    """
    Spectrogram plotting function
    
    params:::
    groups: a list of dictionaries
    vmin: min value of color plot
    vmax: max value of color plot
    mode: what data from "groups" to use; could be: "tfr", "tfr_db", "tfr_logratio","tfr_mean",
        "tfr_zlogratio","tfr_zscore","tfr_percent"; 
        zscore = subtracting the mean of baseline values and dividing by the standard deviation of baseline values,
        zlogratio = dividing by the mean of baseline values, taking the log, and dividing by the standard deviation of log baseline values
    cmap: cmap (if not chosen, will end up being rainbow "jet" for regular tfr or red/blue for baseline-normed)
    figW: fig width
    figH: fig height
    save: to save or not to save
    output_dir: save dir
    """
    plt.rcParams['svg.fonttype'] = 'none' 
    fig, ax = plt.subplots(1,len(groups),figsize=(figW,figH),dpi=300)
    
    fig.suptitle(mode)

    #set colormap
    if cmap==None:
        if ((mode=="tfr") or (mode=="tfr_db")):
            cmap="jet"
        else:
            cmap="seismic"

    #set up for db conversion if mode is "tfr_db"
    if mode=="tfr_db":
        mode="tfr"
        db=True
    else:
        db=False

    for g in range(len(groups)):
        if db==False:
            logratios=[i.data for i in groups[g][mode]]
        elif db==True:
            logratios=[10 * np.log10(i.data) for i in groups[g][mode]]
        logratio_mean = np.mean(np.array(logratios).squeeze(1),axis=0)
        logratios.clear()
                
        # The actual plotting function
        extent = [groups[g][mode][0].times[0]/60, groups[g][mode][0].times[-1]/60, 0, 55]
        powerplot = ax[g].imshow(logratio_mean, aspect='auto', origin='lower', cmap=cmap, extent=extent, vmax=vmax, vmin=vmin) 
        ax[g].set_ylim([0,63]) 
       
        if g==0:
            ax[g].plot([5,10],[58,58],linewidth=2.5,color="dodgerblue",label="Infusion")
            # Check if 'FUS' is in the group label
            if ('-FUS' not in groups[g]['label']) and ('mg/kg' not in groups[g]['label']):
                ax[g].plot([7.5,10],[61,61],linewidth=2.5,color="black",label="Sonication")
        else:
            ax[g].plot([5,10],[58,58],linewidth=2.5,color="dodgerblue")
            # Check if 'FUS' is in the group label
            if ('-FUS' not in groups[g]['label']) and ('mg/kg' not in groups[g]['label']):
                ax[g].plot([7.5,10],[61,61],linewidth=2.5,color="black")
        
               
        ax[g].spines['top'].set_visible(False)
        ax[g].axhline(100,color="black",linewidth=0.75)
        ax[g].set_xlabel('Time (min)', fontsize=14)
        if g == 0:  # This is the first plot
            ax[g].set_ylabel('Frequency (Hz)', fontsize=14)
        ax[g].set_title(groups[g]["label"])
        # Adjusting the font sizes
        ax[g].tick_params(axis='both', which='major', labelsize=14) # Choose the font size here
        
        fontsize = 14 #for the colorbar
        
        #What to call the colorbar lol
        if ((mode=="tfr") and (db==True)):
            cbar_label='Power (dB)'
        elif ((mode=="tfr") and (db==False)):
            cbar_label='Power (μV²/Hz)'
        elif mode=="tfr_zlogratio":
            cbar_label='Z-log ratio to baseline'
        elif mode=="tfr_logratio":
            cbar_label='Log ratio to baseline'
        elif mode=="tfr_zscore":
            cbar_label='Z-score'
        elif mode=="tfr_percent":
            cbar_label='Percent Δ from baseline'
        elif mode=="tfr_mean":
            cbar_label='Δ from baseline mean'
        else:
            cbar_label=""
        if g==len(groups)-1:
            #This code creates a space(divider) for colorbar next to each subplot
            divider = make_axes_locatable(ax[g])
            cax = divider.append_axes("right", size="5%", pad=0.15)            
            cbar = plt.colorbar(powerplot, cax=cax, label=cbar_label)
            cbar.ax.tick_params(labelsize=14)   # set your label size here
            cbar.set_label(cbar_label, size=fontsize)
        else:  
            divider = make_axes_locatable(ax[g])
            cax = divider.append_axes("right", size="5%", pad=0.15)
            cax.axis('off')  # switch off the extra axis for non-last plots
    
    fig.legend()
    plt.tight_layout()
    if save==True:
        plt.savefig(output_dir + mode + ".svg", format='svg', dpi=500)
    plt.show()




def moving_average(x, w):
    """
    run of the mill application of np.convolve

    params:::
    x: 1D data
    w: convolving window length
    """
    return np.convolve(x, np.ones(w), 'valid') / w




def fmt(x, pos):
    a, b = '{:.2e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)



#Plots PSD over frequency
def plot_spectral_density(groups, group_names, mode, time_bins, colors, 
                          save=False, output_dir=None, ylim_min=None, ylim_max=None):

    # Create the figure and axes objects
    plt.rcParams['svg.fonttype'] = 'none' 
    fig, axs = plt.subplots(1, len(time_bins), sharey=True, figsize=(5*len(time_bins),2.5), dpi=300) #horizontal

    for ind, (t_start, t_end) in enumerate(time_bins):
        for g in range(len(groups)):
            logratios=[i.data for i in groups[g][mode]]
            logratio_mean = np.mean(np.array(logratios).squeeze(1),axis=0)
            logratio_sem = stats.sem(np.array(logratios).squeeze(1), axis=0)
            logratios.clear()
            freqs = groups[g][mode][0].freqs

            time_mask = np.logical_and(groups[g][mode][0].times/60 >= t_start, groups[g][mode][0].times/60 <= t_end)

            mean_psd = np.mean(logratio_mean[:, time_mask], axis=1)
            sem_psd = np.mean(logratio_sem[:, time_mask], axis=1) 

            axs[ind].plot(freqs, mean_psd, label=group_names[g], color=colors[g])

            # Plot standard error as a shaded region
            axs[ind].set_xlabel('Frequency (Hz)')
            
            # Set y-axis limit if specified
            if ylim_min is not None and ylim_max is not None:
                axs[ind].set_ylim(ylim_min, ylim_max)
            
            if mode=="tfr_zscore":
                axs[ind].set_ylabel('Z-Score')
            elif mode=="tfr_percent":
                axs[ind].set_ylabel('% Change')
            elif mode=="tfr_mean":
                axs[ind].set_ylabel('Δ from baseline mean')
            else:
                cbar_label=""
            axs[ind].set_title(f'{t_start}-{t_end} Mins')
    plt.tight_layout()  

    if save:
        if output_dir is None:
            output_dir = os.getcwd()  # Use current working directory if none is provided

        fig.savefig(os.path.join(output_dir, 'spectral_density_plot.svg'))

    else:
        plt.show()
        

#Calculate the AUC for a specific time frame and a specific frequency
def calculate_avg_auc(mode, groups, group_titles, low_freq, high_freq, auc, t_start, t_end, print_data=False):
    avg_auc_list = []
    all_auc_ind = []

    for g in range(len(groups)):
        time_mask = np.logical_and(groups[g][mode][0].times/60 >= t_start, groups[g][mode][0].times/60 <= t_end)
        logratios=[i.data for i in groups[g][mode]]
        freqs = groups[g][mode][0].freqs
        idx = np.where((freqs >= low_freq) & (freqs <= high_freq))

        auc_list = []
        for i in range(len(logratios)):
            logratio_data = logratios[i].reshape(-1, logratios[i].shape[-1])
            data_psd = np.mean(logratio_data[:, time_mask], axis=1)
            ind = auc(freqs[idx], data_psd[idx])  #x, y
            auc_list.append(ind)
                       
        avg_auc = np.mean(auc_list)
        avg_auc_list.append(avg_auc)
        all_auc_ind.append(auc_list)
        
        if print_data == True: 
            print(f"group = {auc_list}" )

    return avg_auc_list, all_auc_ind


def plot_auc(groups, avg_aucs, all_aucs, group_titles, colors, save=False, output_dir=None):
    plt.rcParams['svg.fonttype'] = 'none' 
    
    x_pos = np.arange(len(groups))

    for i in range(len(groups)):
        # Add color to the bar
        plt.bar(x_pos[i], avg_aucs[i], align='center', color=colors[i], alpha=0.7)

        # Overlay individual datapoints
        y = all_aucs[i]
        x = [i] * len(y)  # All x-values for a group are the same.
        plt.scatter(x, y, color=colors[i], alpha=0.5)  # Add color to the scatterplot points

    # Add labels / title and show the plot
    plt.xticks(x_pos, group_titles)
    plt.ylabel('AUC')
    plt.title(f'AUC from {t_start}-{t_end} mins for {low_freq}-{high_freq} Hz')

    if save==True:
        plt.savefig(output_dir + str(t_start) + "to" + str(t_end) + "mins_at_" + str(low_freq) + str(high_freq) + "Hz" + ".svg", format='svg', dpi=500)
        
    plt.show()
    
    
def run_stats(groups, group_titles):
    # Collect results in a DataFrame
    col_names = ["Group1", "Group2", "Mean Diff", "p-adj", "Lower", "Upper", "Reject Null"]
    results_df = pd.DataFrame(columns = ["Statistic", "P-value"])
        
    # Perform one-way ANOVA
    stat, pval = stats.f_oneway(*groups)
    print('ANOVA statistic:', stat)
    print('p-value:', pval)
    results_df.loc[0] = [stat, pval]

    # Perform and print Tukey HSD results
    data_all = np.concatenate(groups)
    group_labels = []
    for i, group in enumerate(groups):
        group_labels += [group_titles[i]] * len(group)
    tukey_results = pairwise_tukeyhsd(data_all, group_labels)
    print(tukey_results)
    summary = tukey_results.summary()
    summary_df = pd.DataFrame(summary[1:], columns=col_names)
    
    stats_df = pd.concat([results_df, summary_df], axis=1, keys=['ANOVA', 'Tukey HSD'])

    return stats_df

def calculate_hedges_g(data1, data2):
    # get means
    mean1, mean2 = np.mean(data1), np.mean(data2)
    
    # Corrected pooled standard deviation calculation
    n1, n2 = len(data1), len(data2)
    sd1, sd2 = np.std(data1, ddof=1), np.std(data2, ddof=1)
    s_pooled = sqrt(((n1 - 1) * sd1 ** 2 + (n2 - 1) * sd2 ** 2) / (n1 + n2 - 2))
    
    # Delta calculation
    delta = (mean1 - mean2) / s_pooled
    
    # Correcting factor calculation
    j = 1 - 3 / (4 * (n1 + n2) - 9)
    
    # Hedges' g
    g = j * delta
    return g

Note: Get info for all of the functions using "help"

In [ ]:
# help(group_eeg_processing)

## Optional: for running full notebook all at once, set paths and options here

In [ ]:
# all
filter_type="mne"
denoise=True 
save=True
pickle_save_dir = "./pickle/"

# mPFC
mPFC_uncaging_figures_dir="./output/mPFC_uncaging/"
mPFC_dr_figures_dir="./output/mPFC_DR/"

# RSC
RSC_uncaging_figures_dir="./output/RSC_uncaging/"
RSC_dr_figures_dir="./output/RSC_DR/"

# mPFC Uncaging

### Set up dictionaries for depositing data

In [ ]:
# Set directory where the data are stored
root_directory="./SourceData"

# Set up the dictionaries where we will be dumping all data, labels, calculations, etc
LipKFUS = {"directory": root_directory+"/mPFC/LipKFUS/"}
LipKnoFUS = {"directory": root_directory+"/mPFC/LipKonly/"}
FreeKnoFUS = {"directory": root_directory+"/mPFC/Ketamine/"}
SalFUS = {"directory": root_directory+"/mPFC/SalineFUS/"}

# Add labels
LipKFUS["label"]="LipK +FUS"
LipKnoFUS["label"]="LipK -FUS" 
FreeKnoFUS["label"]="FreeK -FUS" 
SalFUS["label"]="Sal +FUS" 

# Add filepaths
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
for g in range(len(groups)):
    file_paths = []
    data_dir = groups[g]["directory"]
    data_files = [i for i in os.listdir(data_dir) if "._" not in i]
    
    for file in data_files:
        full_path = os.path.join(data_dir, file)  # Create full path to every csv file
        file_paths.append(full_path)  # Add each csv file path to list

    groups[g]["file_paths"] = file_paths  # Save paths in each group

### Option 1: Preprocess for time-frequency analysis

In [ ]:
#Iterate through groups to apply bandpass and notch filters, compute power spectral density, compute time frequency representation, and compute
filter_type="mne"
denoise=True
srate = 1000.0
low_cutoff = 1
high_cutoff = 200
notch_width = 1
thresh=1

for g in range(len(groups)):
    groups[g] = group_eeg_processing(groups[g],
                                 srate,
                                 "single_epoch",
                                 low_cutoff=low_cutoff,
                                 high_cutoff=high_cutoff,
                                 notch_width=notch_width,
                                 denoise=denoise,
                                 filter_type=filter_type,
                                 denoise_extent_threshold=thresh)
    
    print("DONE WITH GROUP " + str(g+1) + " OF " + str(len(groups)))

### Option 2: load from pickle

In [ ]:
# pickle_dir = pickle_save_dir

# filename = pickle_dir + 'mPFC_SalFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     SalFUS = pickle.load(file)
#     print("Done loading pickle 1 of 4")
    
# filename = pickle_dir + 'mPFC_LipKnoFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     LipKnoFUS = pickle.load(file)
#     print("Done loading pickle 2 of 4")
    
# filename = pickle_dir + 'mPFC_LipKFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     LipKFUS = pickle.load(file)
#     print("Done loading pickle 3 of 4")
    
# filename = pickle_dir + 'mPFC_FreeKnoFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     FreeKnoFUS = pickle.load(file)
#     print("Done loading pickle 4 of 4")

### Full session spectrogram

In [ ]:
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]

output_dir=mPFC_uncaging_figures_dir
save = False
figW=12
figH=2.5

# plot_spectrogram(groups,
#                  10**-9.5,10**-7.6,
#                  "tfr",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  10 * np.log10(10**-10),10 * np.log10(10**-7),
#                  "tfr_db",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1,1,
#                  "tfr_logratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1.5,1.5,
#                  "tfr_zlogratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

plot_spectrogram(groups,
                 -200,200,
                 "tfr_percent",
                 figW=figW,figH=figH,save=save,output_dir=output_dir)

### Percent Change over Frequency

In [ ]:
output_dir=mPFC_uncaging_figures_dir
save = False
decim = 10
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
group_names = ['Sal FUS(+)', 'Ket FUS(-)', 'SonoKet FUS(-)', 'SonoKet FUS(+)'] 
colors = ['darkgrey', 'black', 'mistyrose', 'indianred']
time_bins = [(7.5, 10), (10, 25), (25, 45)]
mode = "tfr_percent"

plot_spectral_density(groups=groups, group_names=group_names, mode=mode, time_bins=time_bins, colors=colors, save=save, output_dir=output_dir, ylim_min=-19, ylim_max=40)

### Area Under the Curve and Stats

In [ ]:
output_dir = mPFC_uncaging_figures_dir
print_data=False
mode = "tfr_percent"
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
group_titles = ["SalFUS", "FreeKnoFUS", "LipKnoFUS", "LipKFUS"] 
colors = ['darkgrey', 'black', 'mistyrose', 'indianred']
time_bins = [(7.5, 10), (10, 25), (25, 45)]
freq_bins = [(1,4),(4,8),(8,15),(15,25),(25,55)]

all_stats = []
all_hedges = []

for t_start, t_end in time_bins:
    for low_freq, high_freq in freq_bins:
        avg_aucs, all_aucs = calculate_avg_auc(mode, groups, group_titles, low_freq, high_freq, auc, t_start, t_end, print_data)       
        stats_df = run_stats(all_aucs, group_titles)

        # Add the time_bin and freq_bin info to the DataFrame
        stats_df['time_bin_start'] = t_start
        stats_df['time_bin_end'] = t_end
        stats_df['freq_bin_low'] = low_freq
        stats_df['freq_bin_high'] = high_freq
        all_stats.append(stats_df)
        
        group_pairs_ind = list(itertools.combinations(range(len(all_aucs)), 2))    
        for i, j in group_pairs_ind:
            hedges_g = calculate_hedges_g(all_aucs[i], all_aucs[j])
            print(f"Hedges' g for groups {i} and {j} at time bin {t_start}-{t_end} and freq bin {low_freq}-{high_freq}: {hedges_g}")
            # create a DataFrame to hold the Hedges' g results and store in the all_hedges_dfs list
            df_hedges = pd.DataFrame({
                'Group1': [group_titles[i]],
                'Group2': [group_titles[j]],
                'Hedges_g': [hedges_g],
                'time_bin_start': [t_start],
                'time_bin_end': [t_end],
                'freq_bin_low': [low_freq],
                'freq_bin_high': [high_freq]
            })
            all_hedges.append(df_hedges)    
        
        plot_auc(groups, avg_aucs, all_aucs, group_titles, colors, save=False, output_dir=output_dir)

# all_stats_combined = pd.concat(all_stats, ignore_index=True)
# all_hedges_combined = pd.concat(all_hedges, ignore_index=True)
# all_stats_combined.to_csv(output_dir + 'stats.csv', index=False)
# all_hedges_combined.to_csv(output_dir + "hedges.csv", index=False)

### Save as pickles

In [ ]:
# pickle_dir = pickle_save_dir

# with open(pickle_dir+'mPFC_SalFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(SalFUS, file)
# with open(pickle_dir+'mPFC_LipKnoFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(LipKnoFUS, file)
# with open(pickle_dir+'mPFC_LipKFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(LipKFUS, file)
# with open(pickle_dir+'mPFC_FreeKnoFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(FreeKnoFUS, file)

## mPFC Dose-response - full session analysis

### Set up dictionaries for depositing data

In [ ]:
# Set directory where the data are stored
root_directory="./SourceData"

# Set up the dictionaries where we will be dumping all data, labels, calculations, etc
FreeK0500 = {"directory": root_directory+"/DoseResponse_mPFC/Ket_inf0.5/"}
FreeK0750 = {"directory": root_directory+"/DoseResponse_mPFC/Ket_inf0.75/"}
FreeK1000 = {"directory": root_directory+"/DoseResponse_mPFC/Ket_inf1/"}
FreeK1500 = {"directory": root_directory+"/DoseResponse_mPFC/Ket_inf1.5/"}

# Add labels
FreeK0500["label"]="0.5 mg/kg Ket"
FreeK0750["label"]="0.75 mg/kg Ket"
FreeK1000["label"]="1.0 mg/kg Ket"
FreeK1500["label"]="1.5 mg/kg Ket"

# Add filepaths
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
for g in range(len(groups)):
    file_paths = []
    data_dir = groups[g]["directory"]
    data_files = [i for i in os.listdir(data_dir) if "._" not in i]
    
    for file in data_files:
        full_path = os.path.join(data_dir, file)  # Create full path to every csv file
        file_paths.append(full_path)  # Add each csv file path to list

    groups[g]["file_paths"] = file_paths  # Save paths in each group

### Option 1: Preprocess for time-frequency analysis

In [ ]:
#Iterate through groups to apply bandpass and notch filters, compute power spectral density, compute time frequency representation, and compute
filter_type="mne" 
denoise=True
srate = 1000.0
low_cutoff = 1
high_cutoff = 200
notch_width = 1
thresh=1

for g in range(len(groups)):
    groups[g] = group_eeg_processing(groups[g],
                                 srate,
                                 "single_epoch",
                                 low_cutoff=low_cutoff,
                                 high_cutoff=high_cutoff,
                                 notch_width=notch_width,
                                 denoise=denoise,
                                 filter_type=filter_type,
                                 denoise_extent_threshold=thresh)
    print("DONE WITH GROUP " + str(g+1) + " OF " + str(len(groups)))

### Option 2: Load from pickle

In [ ]:
# pickle_dir = pickle_save_dir

# filename = pickle_dir + 'mPFC_FreeK0500.pkl'
# with open(filename, 'rb') as file:
#     FreeK0500 = pickle.load(file)
#     print("Done loading pickle 1 of 4")
    
# filename = pickle_dir + 'mPFC_FreeK0750.pkl'
# with open(filename, 'rb') as file:
#     FreeK0750 = pickle.load(file)
#     print("Done loading pickle 2 of 4")
    
# filename = pickle_dir + 'mPFC_FreeK1000.pkl'
# with open(filename, 'rb') as file:
#     FreeK1000 = pickle.load(file)
#     print("Done loading pickle 3 of 4")
    
# filename = pickle_dir + 'mPFC_FreeK1500.pkl'
# with open(filename, 'rb') as file:
#     FreeK1500 = pickle.load(file)
#     print("Done loading pickle 4 of 4")

### Full session spectrogram

In [ ]:
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
output_dir=mPFC_dr_figures_dir
save=False
figW=12
figH=2.5

# plot_spectrogram(groups,
#                  10**-9.5,10**-7.6,
#                  "tfr",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  10 * np.log10(10**-10),10 * np.log10(10**-7),
#                  "tfr_db",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1,1,
#                  "tfr_logratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1.5,1.5,
#                  "tfr_zlogratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

plot_spectrogram(groups,
                 -200,200,
                 "tfr_percent",
                 figW=figW,figH=figH,save=save,output_dir=output_dir)

### Percent Change over Frequency

In [ ]:
output_dir=mPFC_dr_figures_dir
save=False
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
group_names = ['0.5 mg/kg', '0.75 mg/kg', '1.0 mg/kg', '1.5 mg/kg'] 
colors = ['lightgrey', 'darkgrey', 'dimgray', 'black']
mode = "tfr_percent"
time_bins = [(7.5, 10), (10, 25), (25, 45)]
decim = 10

plot_spectral_density(groups=groups, group_names=group_names, mode="tfr_percent", time_bins=time_bins, colors=colors, save=save, output_dir=output_dir, ylim_min=-40, ylim_max=150)

### Area Under the Curve and Stats

In [ ]:
output_dir=mPFC_dr_figures_dir
print_data=False
mode = "tfr_percent"
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
group_titles = ['0.5 mg/kg', '0.75 mg/kg', '1.0 mg/kg', '1.5 mg/kg'] 
colors = ['lightgrey', 'darkgrey', 'dimgray', 'black']
time_bins = [(7.5, 10),(10, 25),(25, 45)]
freq_bins = [(1,4),(4,8),(8,15),(15,25),(25,55)]

all_stats = []
all_hedges = []

for t_start, t_end in time_bins:
    for low_freq, high_freq in freq_bins:
        avg_aucs, all_aucs = calculate_avg_auc(mode, groups, group_titles, low_freq, high_freq, auc, t_start, t_end, print_data)       
        stats_df = run_stats(all_aucs, group_titles)

        # Add the time_bin and freq_bin info to the DataFrame
        stats_df['time_bin_start'] = t_start
        stats_df['time_bin_end'] = t_end
        stats_df['freq_bin_low'] = low_freq
        stats_df['freq_bin_high'] = high_freq
        all_stats.append(stats_df)
        
        group_pairs_ind = list(itertools.combinations(range(len(all_aucs)), 2))    
        for i, j in group_pairs_ind:
            hedges_g = calculate_hedges_g(all_aucs[i], all_aucs[j])
            print(f"Hedges' g for groups {i} and {j} at time bin {t_start}-{t_end} and freq bin {low_freq}-{high_freq}: {hedges_g}")
            # create a DataFrame to hold the Hedges' g results and store in the all_hedges_dfs list
            df_hedges = pd.DataFrame({
                'Group1': [group_titles[i]],
                'Group2': [group_titles[j]],
                'Hedges_g': [hedges_g],
                'time_bin_start': [t_start],
                'time_bin_end': [t_end],
                'freq_bin_low': [low_freq],
                'freq_bin_high': [high_freq]
            })
            all_hedges.append(df_hedges)    
        
        plot_auc(groups, avg_aucs, all_aucs, group_titles, colors, save=False, output_dir=output_dir)

# all_stats_combined = pd.concat(all_stats, ignore_index=True)
# all_hedges_combined = pd.concat(all_hedges, ignore_index=True)
# all_stats_combined.to_csv(output_dir + 'stats.csv', index=False)
# all_hedges_combined.to_csv(output_dir + "hedges.csv", index=False)

### Save as pickles

In [ ]:
# pickle_dir = pickle_save_dir

# with open(pickle_dir+'mPFC_FreeK0500.pkl', 'wb') as file:
#     pickle.dump(FreeK0500, file)
# with open(pickle_dir+'mPFC_FreeK0750.pkl', 'wb') as file:
#     pickle.dump(FreeK0750, file)
# with open(pickle_dir+'mPFC_FreeK1000.pkl', 'wb') as file:
#     pickle.dump(FreeK1000, file)
# with open(pickle_dir+'mPFC_FreeK1500.pkl', 'wb') as file:
#     pickle.dump(FreeK1500, file)

# Retrosplenial Cortex Uncaging - full session 

### Set up dictionaries for depositing data

In [ ]:
# Set directory where the data are stored
root_directory="./SourceData"

# Set up the dictionaries where we will be dumping all data, labels, calculations, etc
LipKFUS = {"directory": root_directory+"/RsC/LipKFUS/"}
LipKnoFUS = {"directory": root_directory+"/RsC/LipKonly/"}
FreeKnoFUS = {"directory": root_directory+"/RsC/Ketamine/"}
SalFUS = {"directory": root_directory+"/RsC/SalineFUS/"}

# Add labels
LipKFUS["label"]="LipK +FUS"
LipKnoFUS["label"]="LipK -FUS" 
FreeKnoFUS["label"]="FreeK -FUS" 
SalFUS["label"]="Sal +FUS" 

# Add filepaths
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
for g in range(len(groups)):
    file_paths = []
    data_dir = groups[g]["directory"]
    data_files = [i for i in os.listdir(data_dir) if "._" not in i]
    
    for file in data_files:
        full_path = os.path.join(data_dir, file)  # Create full path to every csv file
        file_paths.append(full_path)  # Add each csv file path to list

    groups[g]["file_paths"] = file_paths  # Save paths in each group

### Option 1: Preprocess for time-frequency analysis

In [ ]:
#Iterate through groups to apply bandpass and notch filters, compute power spectral density, compute time frequency representation, and compute
filter_type="mne"
denoise=True
srate = 1000.0
low_cutoff = 1
high_cutoff = 200
notch_width = 1
thresh=1

for g in range(len(groups)):
    groups[g] = group_eeg_processing(groups[g],
                                 srate,
                                 "single_epoch",
                                 low_cutoff=low_cutoff,
                                 high_cutoff=high_cutoff,
                                 notch_width=notch_width,
                                 denoise=denoise,
                                 filter_type=filter_type,
                                 denoise_extent_threshold=thresh)
    print("DONE WITH GROUP " + str(g+1) + " OF " + str(len(groups)))

### Option 2: Load from pickle

In [ ]:
# pickle_dir = pickle_save_dir

# filename = pickle_dir + 'RSC_SalFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     SalFUS = pickle.load(file)
#     print("Done loading pickle 1 of 4")
    
# filename = pickle_dir + 'RSC_LipKnoFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     LipKnoFUS = pickle.load(file)
#     print("Done loading pickle 2 of 4")
    
# filename = pickle_dir + 'RSC_LipKFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     LipKFUS = pickle.load(file)
#     print("Done loading pickle 3 of 4")
    
# filename = pickle_dir + 'RSC_FreeKnoFUS_single-epoch.pkl'
# with open(filename, 'rb') as file:
#     FreeKnoFUS = pickle.load(file)
#     print("Done loading pickle 4 of 4")

### Full session spectrogram

In [ ]:
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
output_dir=RSC_uncaging_figures_dir
save=False
figW=12
figH=2.5

# plot_spectrogram(groups,
#                  10**-9.5,10**-7.6,
#                  "tfr",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  10 * np.log10(10**-10),10 * np.log10(10**-7),
#                  "tfr_db",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1,1,
#                  "tfr_logratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1.5,1.5,
#                  "tfr_zlogratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

plot_spectrogram(groups,
                 -200,200,
                 "tfr_percent",
                 figW=figW,figH=figH,save=save,output_dir=output_dir)

### Percent Change over Frequency

In [ ]:
output_dir=RSC_uncaging_figures_dir
save=False
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
group_names = ['Sal FUS(+)', 'Ket FUS(-)', 'SonoKet FUS(-)', 'SonoKet FUS(+)'] 
colors = ['darkgrey', 'black', 'lightsteelblue', 'cornflowerblue']
mode = "tfr_percent"
decim = 10
time_bins = [(7.5, 10), (10, 25), (25, 45)]  

plot_spectral_density(groups=groups, group_names=group_names, mode=mode, time_bins=time_bins, colors=colors, save=save, output_dir=output_dir)

### Area Under the Curve and Stats

In [ ]:
output_dir = RSC_uncaging_figures_dir
print_data=False
mode = "tfr_percent"
groups = [SalFUS, FreeKnoFUS, LipKnoFUS, LipKFUS]
group_titles = ["SalFUS", "FreeKnoFUS", "LipKnoFUS", "LipKFUS"] 
colors = ['darkgrey', 'black', 'lightsteelblue', 'cornflowerblue']
time_bins = [(7.5, 10), (10, 25), (25, 45)]
freq_bins = [(1,4), (4,8), (8,15), (15,25), (25,55)]

all_stats = []
all_hedges = []

for t_start, t_end in time_bins:
    for low_freq, high_freq in freq_bins:
        avg_aucs, all_aucs = calculate_avg_auc(mode, groups, group_titles, low_freq, high_freq, auc, t_start, t_end, print_data)       
        stats_df = run_stats(all_aucs, group_titles)

        # Add the time_bin and freq_bin info to the DataFrame
        stats_df['time_bin_start'] = t_start
        stats_df['time_bin_end'] = t_end
        stats_df['freq_bin_low'] = low_freq
        stats_df['freq_bin_high'] = high_freq
        all_stats.append(stats_df)
        
        group_pairs_ind = list(itertools.combinations(range(len(all_aucs)), 2))    
        for i, j in group_pairs_ind:
            hedges_g = calculate_hedges_g(all_aucs[i], all_aucs[j])
            print(f"Hedges' g for groups {i} and {j} at time bin {t_start}-{t_end} and freq bin {low_freq}-{high_freq}: {hedges_g}")
            # create a DataFrame to hold the Hedges' g results and store in the all_hedges_dfs list
            df_hedges = pd.DataFrame({
                'Group1': [group_titles[i]],
                'Group2': [group_titles[j]],
                'Hedges_g': [hedges_g],
                'time_bin_start': [t_start],
                'time_bin_end': [t_end],
                'freq_bin_low': [low_freq],
                'freq_bin_high': [high_freq]
            })
            all_hedges.append(df_hedges)    
        
        plot_auc(groups, avg_aucs, all_aucs, group_titles, colors, save=False, output_dir=output_dir)

# all_stats_combined = pd.concat(all_stats, ignore_index=True)
# all_hedges_combined = pd.concat(all_hedges, ignore_index=True)
# all_stats_combined.to_csv(output_dir + 'stats.csv', index=False)
# all_hedges_combined.to_csv(output_dir + "hedges.csv", index=False)

### Save as pickles

In [ ]:
# pickle_dir = pickle_save_dir

# with open(pickle_dir+'RSC_SalFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(SalFUS, file)
# with open(pickle_dir+'RSC_LipKnoFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(LipKnoFUS, file)
# with open(pickle_dir+'RSC_LipKFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(LipKFUS, file)
# with open(pickle_dir+'RSC_FreeKnoFUS_single-epoch.pkl', 'wb') as file:
#     pickle.dump(FreeKnoFUS, file)

## RsC Dose Response - full session analysis

### Set up dictionaries for depositing data

In [ ]:
# Set directory where the data are stored
root_directory="./SourceData"

# Set up the dictionaries where we will be dumping all data, labels, calculations, etc
FreeK0500 = {"directory": root_directory+"/DoseResponse_RSC/Ket0.5/"}
FreeK0750 = {"directory": root_directory+"/DoseResponse_RSC/Ket0.75/"}
FreeK1000 = {"directory": root_directory+"/DoseResponse_RSC/Ket1/"}
FreeK1500 = {"directory": root_directory+"/DoseResponse_RSC/Ket1.5/"}

# Add labels
FreeK0500["label"]="0.5 mg/kg Ket"
FreeK0750["label"]="0.75 mg/kg Ket"
FreeK1000["label"]="1.0 mg/kg Ket"
FreeK1500["label"]="1.5 mg/kg Ket"

# Add filepaths
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]

for g in range(len(groups)):
    file_paths = []
    data_dir = groups[g]["directory"]
    data_files = [i for i in os.listdir(data_dir) if "._" not in i]
    
    for file in data_files:
        full_path = os.path.join(data_dir, file)  # Create full path to every csv file
        file_paths.append(full_path)  # Add each csv file path to list

    groups[g]["file_paths"] = file_paths  # Save paths in each group

### Option 1: Preprocess for time-frequency analysis

In [ ]:
#Iterate through groups to apply bandpass and notch filters, compute power spectral density, compute time frequency representation, and compute
filter_type="mne"
denoise=denoise
srate = 1000.0
low_cutoff = 1
high_cutoff = 200
notch_width = 1
thresh=1

for g in range(len(groups)):
    groups[g] = group_eeg_processing(groups[g],
                                 srate,
                                 "single_epoch",
                                 low_cutoff=low_cutoff,
                                 high_cutoff=high_cutoff,
                                 notch_width=notch_width,
                                 denoise=denoise,
                                 filter_type=filter_type,
                                 denoise_extent_threshold=thresh)
    print("DONE WITH GROUP " + str(g+1) + " OF " + str(len(groups)))

### Option 2: Load from pickle

In [ ]:
# pickle_dir = pickle_save_dir

# filename = pickle_dir + 'RSC_FreeK0500.pkl'
# with open(filename, 'rb') as file:
#     FreeK0500 = pickle.load(file)
#     print("Done loading pickle 1 of 4")
    
# filename = pickle_dir + 'RSC_FreeK0750.pkl'
# with open(filename, 'rb') as file:
#     FreeK0750 = pickle.load(file)
#     print("Done loading pickle 2 of 4")
    
# filename = pickle_dir + 'RSC_FreeK1000.pkl'
# with open(filename, 'rb') as file:
#     FreeK1000 = pickle.load(file)
#     print("Done loading pickle 3 of 4")
    
# filename = pickle_dir + 'RSC_FreeK1500.pkl'
# with open(filename, 'rb') as file:
#     FreeK1500 = pickle.load(file)
#     print("Done loading pickle 4 of 4")

### Full session spectrogram

In [ ]:
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
output_dir=RSC_dr_figures_dir
save=False
figW=12
figH=2.5

# plot_spectrogram(groups,
#                  10**-9.5,10**-7.6,
#                  "tfr",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  10 * np.log10(10**-10),10 * np.log10(10**-7),
#                  "tfr_db",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1,1,
#                  "tfr_logratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

# plot_spectrogram(groups,
#                  -1.5,1.5,
#                  "tfr_zlogratio",
#                  figW=figW,figH=figH,save=save,output_dir=output_dir)

plot_spectrogram(groups,
                 -200,200,
                 "tfr_percent",
                 figW=figW,figH=figH,save=save,output_dir=output_dir)

### Percent Change over Frequency

In [ ]:
output_dir=RSC_dr_figures_dir
save=False
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
group_names = ['0.5 mg/kg', '0.75 mg/kg', '1.0 mg/kg', '1.5 mg/kg'] 
colors = ['lightgrey', 'darkgrey', 'dimgray', 'black']
mode = "tfr_percent"
decim = 10
time_bins = [(7.5, 10), (10, 25), (25,45)]  
plot_spectral_density(groups=groups, group_names=group_names, mode="tfr_percent", time_bins=time_bins, colors=colors, save=save, output_dir=output_dir, ylim_min=-40, ylim_max=150)

### Area Under the Curve and Stats

In [ ]:
output_dir=RSC_dr_figures_dir
print_data=False
mode = "tfr_percent"
groups = [FreeK0500, FreeK0750, FreeK1000, FreeK1500]
group_titles = ['0.5 mg/kg', '0.75 mg/kg', '1.0 mg/kg', '1.5 mg/kg'] 
colors = ['lightgrey', 'darkgrey', 'dimgray', 'black']
time_bins = [(7.5, 10),(10, 25),(25, 45)]
freq_bins = [(1,4),(4,8),(8,15),(15,25),(25,55)]

all_stats = []
all_hedges = []

for t_start, t_end in time_bins:
    for low_freq, high_freq in freq_bins:
        avg_aucs, all_aucs = calculate_avg_auc(mode, groups, group_titles, low_freq, high_freq, auc, t_start, t_end, print_data)       
        stats_df = run_stats(all_aucs, group_titles)

        # Add the time_bin and freq_bin info to the DataFrame
        stats_df['time_bin_start'] = t_start
        stats_df['time_bin_end'] = t_end
        stats_df['freq_bin_low'] = low_freq
        stats_df['freq_bin_high'] = high_freq
        all_stats.append(stats_df)
        
        group_pairs_ind = list(itertools.combinations(range(len(all_aucs)), 2))    
        for i, j in group_pairs_ind:
            hedges_g = calculate_hedges_g(all_aucs[i], all_aucs[j])
            print(f"Hedges' g for groups {i} and {j} at time bin {t_start}-{t_end} and freq bin {low_freq}-{high_freq}: {hedges_g}")
            # create a DataFrame to hold the Hedges' g results and store in the all_hedges_dfs list
            df_hedges = pd.DataFrame({
                'Group1': [group_titles[i]],
                'Group2': [group_titles[j]],
                'Hedges_g': [hedges_g],
                'time_bin_start': [t_start],
                'time_bin_end': [t_end],
                'freq_bin_low': [low_freq],
                'freq_bin_high': [high_freq]
            })
            all_hedges.append(df_hedges)    
        
        plot_auc(groups, avg_aucs, all_aucs, group_titles, colors, save=False, output_dir=output_dir)

# all_stats_combined = pd.concat(all_stats, ignore_index=True)
# all_hedges_combined = pd.concat(all_hedges, ignore_index=True)
# all_stats_combined.to_csv(output_dir + 'stats.csv', index=False)
# all_hedges_combined.to_csv(output_dir + "hedges.csv", index=False)

### Save as pickles

In [ ]:
# pickle_dir = pickle_save_dir

# with open(pickle_dir+'RSC_FreeK0500.pkl', 'wb') as file:
#     pickle.dump(FreeK0500, file)
# with open(pickle_dir+'RSC_FreeK0750.pkl', 'wb') as file:
#     pickle.dump(FreeK0750, file)
# with open(pickle_dir+'RSC_FreeK1000.pkl', 'wb') as file:
#     pickle.dump(FreeK1000, file)
# with open(pickle_dir+'RSC_FreeK1500.pkl', 'wb') as file:
#     pickle.dump(FreeK1500, file)